# IMPORT

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader,TensorDataset
import tqdm
import torchvision
from torchvision import transforms

# CONFIG

In [2]:
BATCH_SIZE = 10
EPOCHS = 5
LEARNING_RATE = 1e-3
MOMENTUM = 0.9
LOG_INTERVAL = 10

RANDOM_SEED = 42
torch.backends.cudnn.enabled = False
torch.manual_seed(RANDOM_SEED)

# DATASET AND TRANSFORMATION

In [3]:
import matplotlib
import matplotlib.pyplot as plt

mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X = mnist['data'] / 255.0  # Normalizing to [0,1]
y = mnist['target'].astype(int)
transform = transforms.Normalize((0.1307,), (0.3081,))

X_tensor = torch.tensor(X, dtype=torch.float32).reshape(-1, 1, 28, 28)
X_tensor = transform(X_tensor)  # Áp dụng normalization
y_tensor = torch.tensor(y, dtype=torch.long)


train_img , test_img , train_lbl, test_lbl = train_test_split(X_tensor , y_tensor, test_size = 1/7.0 , random_state = 42)
train_lbl[0:10]
train_img.shape

#image = mnist.data.iloc[4].values.reshape((28,28))
#label =  mnist.target.iloc[4]

#plt.figure(figsize = (15,2))
#imgplot = plt.imshow(image,cmap=plt.cm.binary)
#plt.show()
#print("Label:",label)

torch.Size([60000, 1, 28, 28])

In [4]:
train_dataset = TensorDataset(train_img, train_lbl)
test_dataset = TensorDataset(test_img, test_lbl)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# Example of test case
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples) #enumerate đánh số index cho các phần tử
example_data.shape

torch.Size([10, 1, 28, 28])

# MODEL AND OPTIMIZATION

In [6]:
class CNNModel(torch.nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__() # gọi hàm từ class cha
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1 ,320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x , training=self.training)   
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
    

In [7]:
model = CNNModel()
optimizer = optim.SGD(model.parameters() , lr=LEARNING_RATE, momentum=MOMENTUM)

# TRAINING

In [8]:
train_losses = []
train_counter = []
test_losses  = []
test_counter = [i * len(train_dataset)  for i in range(EPOCHS + 1)]

In [9]:
def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = model(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % LOG_INTERVAL == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(model.state_dict(), '/results/model.pth')
      torch.save(optimizer.state_dict(), '/results/optimizer.pth')

In [10]:
def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = model(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

# PROCESS

In [11]:
test()
for epoch in range(1, EPOCHS + 1):
  train(epoch)
  test()

C:\Users\Admin\AppData\Local\Temp\ipykernel_16492\2607894272.py:16: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 2.2966, Accuracy: 775/10000 (8%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.255293


RuntimeError: Parent directory /results does not exist.